In [1]:
#!pip3 install lxml


In [66]:
import pandas as pd
from sqlalchemy import create_engine
import requests
import lxml.html as lh
from bs4 import BeautifulSoup


In [3]:
# cacao_csv = "Resources/flavors_of_cacao.csv"
# cacao_df = pd.read_csv(cacao_csv, encoding="utf-8")
# cacao_df.head()

In [4]:
chocolate_csv = "Resources/chocolate_2020.csv"
chocolate_df = pd.read_csv(chocolate_csv, encoding ="utf-8")
chocolate_df.head()

,Unnamed: 0,ref,company,company_location,review_date,country_of_bean_origin,specific_bean_origin_or_bar_name,cocoa_percent,rating,counts_of_ingredients,...,cocoa_butter,vanilla,lecithin,salt,sugar,sweetener_without_sugar,first_taste,second_taste,third_taste,fourth_taste
0,0,2454,5150,U.S.A,2019,Madagascar,"Bejofo Estate, batch 1",76.0,3.75,3,...,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,cocoa,blackberry,full body,NaN
1,1,2458,5150,U.S.A,2019,Dominican republic,"Zorzal, batch 1",76.0,3.50,3,...,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,cocoa,vegetal,savory,NaN
2,2,2454,5150,U.S.A,2019,Tanzania,"Kokoa Kamili, batch 1",76.0,3.25,3,...,have_cocoa_butter,have_not_vanila,have_not_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,rich cocoa,fatty,bready,NaN
3,3,797,A. Morin,France,2012,Peru,Peru,63.0,3.75,4,...,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,fruity,melon,roasty,NaN
4,4,797,A. Morin,France,2012,Bolivia,Bolivia,70.0,3.50,4,...,have_cocoa_butter,have_not_vanila,have_lecithin,have_not_salt,have_sugar,have_not_sweetener_without_sugar,vegetal,nutty,NaN,NaN


In [49]:
us_csv = "Resources/usa_chocolate.csv"
us_chocolate = pd.read_csv(us_csv, encoding = "utf-8")
us_chocolate.head()

,COMPANY NAME,CITY,STATE,OWNER/MAKER
0,5150 Chocolate Co.,Delray Beach,Florida,Tyler Levitetz
1,9th and Larkin,San Francisco,California,Lan Phan
2,Acalli,Gretna,Louisiana,Carol Morse
3,Adams Chocolate,Nyack,New York,Adam Berrios
4,Affinity,Sugar Land,Texas,Chris Campbell


In [39]:
url = "http://flavorsofcacao.com/usa_craft_makers.html"
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [42]:
[len(T) for T in tr_elements[:12]]

[1, 9, 9, 9, 9, 1, 2, 1, 1, 4, 4]

In [46]:
col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))


1:"
      
                "


In [54]:
[len(C) for (title,C) in col]


[2]

In [68]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=9:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][3].append(data)
        #Increment i for the next column
        i+=1

IndexError: tuple index out of range

In [59]:
url = "http://flavorsofcacao.com/usa_craft_makers.html"
tables = pd.read_html(url)
print(tables[3])

     COMPANY NAME    CITY    STATE    OWNER/MAKER
0  {COMPANY_NAME}  {CITY}  {STATE}  {OWNER/MAKER}


In [71]:
url = "http://flavorsofcacao.com/usa_craft_makers.html"
content = requests.get(url).text

soup = BeautifulSoup(content, "lxml")
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <!-- InstanceBegin template="/Templates/Logo Button Template Final.dwt" codeOutsideHTMLIsLocked="false" -->
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <!-- InstanceBeginEditable name="doctitle" -->
  <title>
   Flavors of Cacao - USA Craft Makers
  </title>
  <!-- InstanceEndEditable -->
  <style type="text/css">
   body {
	background-color: #FFF;
}
  </style>
  <link href="header.css" rel="stylesheet" type="text/css"/>
  <style type="text/css">
   a:link {
	color: #300;
	text-decoration: none;
}
a:visited {
	text-decoration: none;
	color: #300;
}
a:hover {
	text-decoration: none;
	color: #285c93
}
a:active {
	text-decoration: none;
	color: #300;
}
.styleone:link {
	font-weight: normal;
	color: #300;
	text-decoration: none;
	font-size: 14px;
}
.styleone:visited {
	font-weight: normal;

In [90]:
gdp_table = soup.find_all("div", attrs={"spry:region": "ds3"})
gdp_table_data = gdp_table.find_all("tr")  # contains 2 rows

# Get all the headings of Lists
headings = []
for td in gdp_table_data[0].find_all("td"):
    # remove any newlines and extra spaces from left and right
    headings.append(td.b.text.replace('\n', ' ').strip())

print(headings)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [7]:
type(tables)


list

In [55]:
r = requests.get('http://flavorsofcacao.com/usa_craft_makers.html')
pd.read_html(r.text)[3]

,COMPANY NAME,CITY,STATE,OWNER/MAKER
0,{COMPANY_NAME},{CITY},{STATE},{OWNER/MAKER}


In [69]:
tables = pd.read_html("http://flavorsofcacao.com/usa_craft_makers.html", header = 0, keep_default_na=False
# headings = ['COMPANY NAME', 'CITY', 'STATE', 'OWNER/MAKER']
headings = []
for table in tables:
    current_headings = table.columns.values[:4]
    if len(current_headings) != len(headings):
        continue
    if all(current_headings == headings):
        break

# table['Country name'].replace({r'.*!(.*)': r'\1'}, regex=True, inplace=True)
# table[headings].to_csv('test.txt', sep=';', header=False, index=False)


SyntaxError: invalid syntax (<ipython-input-69-db82cfbac8bf>, line 3)

In [25]:
df= tables[3]
df.columns = ['COMPANY NAME', 'CITY', 'STATE', 'OWNER/MAKER']
df

,COMPANY NAME,CITY,STATE,OWNER/MAKER
0,{COMPANY_NAME},{CITY},{STATE},{OWNER/MAKER}


In [92]:
c = requests.get('http://flavorsofcacao.com/usa_craft_makers.html')
soup = BeautifulSoup(c.content, "lxml")
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print(df[0].to_json(orient='records'))

[{"0":null,"1":null,"2":null,"3":null,"4":null,"5":null,"6":null,"7":null,"8":null,"9":null,"10":null},{"0":"Home","1":"Databases","2":"Review Guide","3":"Experiencing Chocolate","4":"Labeling","5":"Flavor","6":"Projects","7":"Contact","8":"Manhattan Chocolate Society","9":null,"10":null},{"0":null,"1":"Chocolate Bar Ratings","2":null,"3":null,"4":null,"5":null,"6":"Interviews","7":null,"8":null,"9":null,"10":null},{"0":null,"1":"USA Craft Makers","2":null,"3":null,"4":null,"5":null,"6":"Craft of Bean to Bar Project","7":null,"8":null,"9":null,"10":null},{"0":null,"1":"Canada Craft Makers","2":null,"3":null,"4":null,"5":null,"6":"Packaging Study","7":null,"8":null,"9":null,"10":null}]
